Notebook to mess around with LNG Price Forecasting


In [81]:
# !pip install keras
!conda install tensorflow
# %conda install -c conda-forge tensorflow 

# !pip install sklearn

^C


In [55]:
import seaborn


In [56]:
import pandas as pd
import numpy as np 
import sklearn 
# import keras 

import datetime

In [57]:
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

Building the dataset
---
---

Loading the data
---

In [58]:
## Loading trade data. Need to calculate 

trades = pd.read_excel('Gas_Data/GSH Historical Trades.xlsx')
trades = trades.replace(to_replace = 'WAL Non-netted', value = 'WAL')
trades.tail()

,TRADE_DATE,PRODUCT,LOCATION,TRADE_PRICE,DAILY_QTY_GJ,START_DATE,END_DATE,MANUAL_TRADE
18580,2022-11-30,Custom,WAL,24.50,4000.0,2022-12-14,2022-12-18,Yes
18581,2022-11-30,BalanceOfDay,WAL,23.25,5000.0,2022-11-30,2022-11-30,Yes
18582,2022-11-30,BalanceOfDay,WAL,23.00,1000.0,2022-11-30,2022-11-30,Yes
18583,2022-11-30,BalanceOfDay,WAL,23.00,5000.0,2022-11-30,2022-11-30,Yes
18584,2022-11-30,DayAhead,WAL,23.00,15000.0,2022-12-01,2022-12-01,Yes


In [59]:
flow_storage = pd.read_csv('Gas_Data/GasBBActualFlowStorage.CSV')
flow_storage.head()
flow_storage.rename(columns = {'GasDate':'Gasdate'}, inplace= True)

In [60]:
flow_storage[(flow_storage['FacilityName'] == 'PVP') ]
## & (flow_storage['GasDate'] == '2022/11/30')

,Gasdate,FacilityName,FacilityId,FacilityType,Demand,Supply,TransferIn,TransferOut,HeldInStorage,CushionGasStorage,State,LocationName,LocationId,LastUpdated
26126,2019/06/03,PVP,580070,PIPE,5.263,0.0,0.00,0.0,NaN,NaN,NT,Regional - NT,590017,2019/06/04 13:38:03
26243,2019/06/04,PVP,580070,PIPE,5.194,0.0,0.00,0.0,NaN,NaN,NT,Regional - NT,590017,2019/06/05 10:05:03
26360,2019/06/05,PVP,580070,PIPE,5.214,0.0,0.00,0.0,NaN,NaN,NT,Regional - NT,590017,2019/06/06 10:05:03
26477,2019/06/06,PVP,580070,PIPE,5.593,0.0,0.00,0.0,NaN,NaN,NT,Regional - NT,590017,2019/06/07 10:05:03
26594,2019/06/07,PVP,580070,PIPE,5.338,0.0,0.00,0.0,NaN,NaN,NT,Regional - NT,590017,2019/06/08 10:05:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190788,2022/12/09,PVP,580070,PIPE,3.959,0.0,2.00,0.0,NaN,NaN,NT,Regional - NT,590017,2022/12/10 08:05:02
190918,2022/12/10,PVP,580070,PIPE,4.084,0.0,2.00,0.0,NaN,NaN,NT,Regional - NT,590017,2022/12/11 08:05:01
191048,2022/12/11,PVP,580070,PIPE,4.007,0.0,2.00,0.0,NaN,NaN,NT,Regional - NT,590017,2022/12/12 08:05:01
191178,2022/12/12,PVP,580070,PIPE,3.984,0.0,1.31,0.0,NaN,NaN,NT,Regional - NT,590017,2022/12/13 08:05:01


In [61]:
noms_fc = pd.read_csv('Gas_Data/GasBBNominationAndForecast.CSV')
noms_fc.head()

,Gasdate,FacilityId,FacilityName,FacilityType,LocationId,LocationName,State,Demand,Supply,TransferIn,TransferOut,LastUpdated
0,2018/09/30,540093,APLNG Pipeline,PIPE,540030,Curtis Island,QLD,1550.0,0.0,0.0,0.0,2018/09/30 15:57:04
1,2018/09/30,540093,APLNG Pipeline,PIPE,590001,Regional - QLD,QLD,0.0,1266.9,534.0,234.9,2018/09/30 15:57:04
2,2018/09/30,540083,Bellevue,PROD,590001,Regional - QLD,QLD,0.0,195.0,0.0,0.0,2018/09/30 17:31:20
3,2018/09/30,540096,Berwyndale to Wallumbilla Pipeline,PIPE,540032,Wallumbilla Hub,QLD,0.0,0.0,40.0,125.0,2018/09/30 19:10:10
4,2018/09/30,540096,Berwyndale to Wallumbilla Pipeline,PIPE,590001,Regional - QLD,QLD,0.0,85.0,0.0,0.0,2018/09/30 19:10:10


In [62]:
noms_fc[(noms_fc['FacilityName'] == 'PVP') & (noms_fc['Gasdate'] == '2022/11/30')]

,Gasdate,FacilityId,FacilityName,FacilityType,LocationId,LocationName,State,Demand,Supply,TransferIn,TransferOut,LastUpdated


Will first merge nomination forecasts and flows and storage data. Merging based on shared gas date, location name, and facility name

In [76]:
df = noms_fc.merge(flow_storage, left_on = ['Gasdate', 'FacilityName', 'LocationName', 'State', 'FacilityType'], \
     right_on = ['Gasdate', 'FacilityName', 'LocationName', 'State', 'FacilityType'], how = 'outer')
print(df.shape)
df['Gasdate']= pd.to_datetime(df['Gasdate'])

# filtered_df = df[['Gasdate', 'FacilityName','FacilityType_x', 'LocationName', 'State_x', 'Demand_x', 'Supply_x', 'TransferIn_x', 'TransferOut_x', 'Demand_y', 'Supply_y', 'TransferIn_y', 'TransferOut_y', 'HeldInStorage' ,'Maximum temperature (Degree C)_x', 'Maximum temperature (Degree C)_y' ]]
filtered_df = df[['Gasdate', 'FacilityName','FacilityType', 'LocationName', 'State', 'Demand_x', 'Supply_x', 'TransferIn_x', 'TransferOut_x', \
'Demand_y', 'Supply_y', 'TransferIn_y', 'TransferOut_y', 'HeldInStorage' ]]

(256917, 21)


In [77]:
filtered_df[filtered_df.FacilityName == 'PVP']


,Gasdate,FacilityName,FacilityType,LocationName,State,Demand_x,Supply_x,TransferIn_x,TransferOut_x,Demand_y,Supply_y,TransferIn_y,TransferOut_y,HeldInStorage
186177,2019-06-03,PVP,PIPE,Regional - NT,NT,NaN,NaN,NaN,NaN,5.263,0.0,0.00,0.0,NaN
186273,2019-06-04,PVP,PIPE,Regional - NT,NT,NaN,NaN,NaN,NaN,5.194,0.0,0.00,0.0,NaN
186369,2019-06-05,PVP,PIPE,Regional - NT,NT,NaN,NaN,NaN,NaN,5.214,0.0,0.00,0.0,NaN
186465,2019-06-06,PVP,PIPE,Regional - NT,NT,NaN,NaN,NaN,NaN,5.593,0.0,0.00,0.0,NaN
186561,2019-06-07,PVP,PIPE,Regional - NT,NT,NaN,NaN,NaN,NaN,5.338,0.0,0.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256841,2022-12-09,PVP,PIPE,Regional - NT,NT,NaN,NaN,NaN,NaN,3.959,0.0,2.00,0.0,NaN
256858,2022-12-10,PVP,PIPE,Regional - NT,NT,NaN,NaN,NaN,NaN,4.084,0.0,2.00,0.0,NaN
256875,2022-12-11,PVP,PIPE,Regional - NT,NT,NaN,NaN,NaN,NaN,4.007,0.0,2.00,0.0,NaN
256892,2022-12-12,PVP,PIPE,Regional - NT,NT,NaN,NaN,NaN,NaN,3.984,0.0,1.31,0.0,NaN


In [43]:
## Function to convert date into correct format for Temp dataframes 
def create_date(year, month, day):
    output = str(year) +"/"+ str(month) +"/"+ str(day)

    return output 

In [44]:
gladstone_maxtemp = pd.read_csv('Weather_Data/gladstone_maxtemp1.csv') ## Gladstone Radar data 
gladstone_maxtemp['Date'] = gladstone_maxtemp.apply(lambda row: create_date(row['Year'], row['Month'], row['Day']), axis = 1)
gladstone_maxtemp['Date']= pd.to_datetime(gladstone_maxtemp['Date'])
gladstone_maxtemp.tail()

,Product code,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Days of accumulation of maximum temperature,Quality,Date
24083,IDCJAC0010,39123,2022,12,9,31.6,1.0,N,2022-12-09
24084,IDCJAC0010,39123,2022,12,10,28.5,1.0,N,2022-12-10
24085,IDCJAC0010,39123,2022,12,11,30.2,1.0,N,2022-12-11
24086,IDCJAC0010,39123,2022,12,12,31.3,1.0,N,2022-12-12
24087,IDCJAC0010,39123,2022,12,13,32.7,1.0,N,2022-12-13


In [45]:
gladstone_maxtemp2 = pd.read_csv('Weather_Data/gladstone_maxtemp2.csv')
gladstone_maxtemp2['Date'] = gladstone_maxtemp2.apply(lambda row: create_date(row['Year'], row['Month'], row['Day']), axis = 1)
gladstone_maxtemp2['Date']= pd.to_datetime(gladstone_maxtemp2['Date'])
# gladstone_maxtemp2

Formatting the data
---

Next we need to create Volume Weighted Average Price 

In [46]:
## Creates VWAP for each location in trades df

trades ['v*p'] = trades['DAILY_QTY_GJ'] * trades['TRADE_PRICE']
tester = trades.groupby(['TRADE_DATE', 'LOCATION'], as_index= False).sum()
tester['VWAP'] = tester['v*p']/tester['DAILY_QTY_GJ']
tester.drop( labels = ['TRADE_PRICE', 'DAILY_QTY_GJ', 'v*p'] , axis = 1, inplace=True)

# tester[tester['TRADE_DATE']>datetime.datetime(2018,12,31)]


RBP = tester[tester ['LOCATION'] == 'RBP'].copy()
RBP.rename(columns={"LOCATION": "RBP", 'VWAP': "VWAP(RBP)"}, inplace= True )

SWQP = tester[tester ['LOCATION'] == 'SWQP'].copy()
SWQP.rename(columns={ "LOCATION": "SWQP", 'VWAP': "VWAP(SWQP)"}, inplace= True )

WAL = tester[tester ['LOCATION'] == 'WAL'].copy()
WAL.rename(columns={ "LOCATION": "WAL", 'VWAP': "VWAP(WAL)"}, inplace= True )



SEQ = tester[tester ['LOCATION'] == 'SEQ'].copy()
SEQ.rename(columns={ "LOCATION": "SEQ", 'VWAP': "VWAP(SEQ)"}, inplace= True )

MSP = tester[tester ['LOCATION'] == 'MSP'].copy()
MSP.rename(columns={ "LOCATION": "MSP", 'VWAP': "VWAP(MSP)"}, inplace= True )

MAP = tester[tester ['LOCATION'] == 'MAP'].copy()
MAP.rename(columns={ "LOCATION": "MAP", 'VWAP': "VWAP(MAP)"}, inplace= True )

MOO = tester[tester ['LOCATION'] == 'MOO Non-netted'].copy()
MOO.rename(columns={ "LOCATION": "MOO", 'VWAP': "VWAP(MOO)"}, inplace= True )



CUL = tester[tester ['LOCATION'] == 'CUL'].copy()
CUL.rename(columns={"LOCATION": "CUL", 'VWAP': "VWAP(CUL)"}, inplace= True )

WIL = tester[tester ['LOCATION'] == 'WIL'].copy()
WIL.rename(columns={"LOCATION": "WIL", 'VWAP': "VWAP(WIL)"}, inplace= True )


df_list = [RBP, SWQP, WAL, SEQ, MSP, MAP, MOO, CUL, WIL]


i = 1

looped_df = RBP


while i < len(df_list):
    print(i)
    looped_df = looped_df.merge(df_list[i], on = 'TRADE_DATE', suffixes= (None,None), how = 'outer' )
    i += 1

looped_df[looped_df['MSP'].notnull()].sort_values('TRADE_DATE', ascending = False)

1
2
3
4
5
6
7
8


,TRADE_DATE,RBP,VWAP(RBP),SWQP,VWAP(SWQP),WAL,VWAP(WAL),SEQ,VWAP(SEQ),MSP,VWAP(MSP),MAP,VWAP(MAP),MOO,VWAP(MOO),CUL,VWAP(CUL),WIL,VWAP(WIL)
2183,2022-10-21,NaN,NaN,NaN,NaN,WAL,14.731959,NaN,NaN,MSP,14.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2120,2022-08-19,NaN,NaN,NaN,NaN,WAL,19.352326,SEQ,18.166667,MSP,17.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2113,2022-08-12,NaN,NaN,NaN,NaN,WAL,18.882407,SEQ,15.175000,MSP,16.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2112,2022-08-11,NaN,NaN,NaN,NaN,WAL,20.180263,NaN,NaN,MSP,16.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2073,2022-07-02,NaN,NaN,NaN,NaN,WAL,41.067692,NaN,NaN,MSP,2.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,2018-07-05,NaN,NaN,NaN,NaN,NaN,NaN,SEQ,9.181818,MSP,9.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2384,2018-07-04,NaN,NaN,NaN,NaN,NaN,NaN,SEQ,9.450000,MSP,9.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2383,2018-06-26,NaN,NaN,NaN,NaN,NaN,NaN,SEQ,10.588000,MSP,10.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2357,2018-05-01,NaN,NaN,NaN,NaN,NaN,NaN,SEQ,7.411207,MSP,7.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We need to rearrange the filtered_df so that we have a single input for each day

In [78]:
filtered_df[(filtered_df['Gasdate'] == '2022-11-30') & (filtered_df['State'] == 'NT')]

,Gasdate,FacilityName,FacilityType,LocationName,State,Demand_x,Supply_x,TransferIn_x,TransferOut_x,Demand_y,Supply_y,TransferIn_y,TransferOut_y,HeldInStorage
163277,2022-11-30,AGP,PIPE,Darwin,NT,29.000,0.000,28.0,16.00,27.006,0.000,24.150,0.000,NaN
163278,2022-11-30,AGP,PIPE,Regional - NT,NT,8.681,34.200,0.0,18.31,9.513,33.817,3.460,16.434,NaN
163286,2022-11-30,BGP,PIPE,Regional - NT,NT,0.331,0.000,0.0,0.00,0.374,0.000,0.000,3.460,NaN
163324,2022-11-30,Mereenie,PROD,Regional - NT,NT,0.000,25.801,0.0,0.00,0.000,25.395,0.000,0.000,NaN
163327,2022-11-30,MRP,PIPE,Regional - NT,NT,0.000,0.000,8.9,0.00,0.000,0.000,8.102,0.000,NaN
163334,2022-11-30,NGP,PIPE,Regional - NT,NT,0.000,0.000,0.0,0.00,0.000,0.000,0.000,0.000,NaN
163339,2022-11-30,Palm Valley,PROD,Regional - NT,NT,0.000,8.800,0.0,0.00,0.000,8.422,0.000,0.000,NaN
163371,2022-11-30,Tanami GP,PIPE,Regional - NT,NT,0.000,0.000,7.5,0.00,7.168,0.000,7.500,0.000,NaN
163387,2022-11-30,WPP,PIPE,Darwin,NT,16.000,44.000,0.0,28.00,13.673,38.146,0.000,24.150,NaN
163388,2022-11-30,Yelcherr,PROD,Regional - NT,NT,0.000,0.000,0.0,0.00,0.000,0.000,0.000,0.000,NaN


In [79]:
filtered_df[(filtered_df['Gasdate'] == '2022-11-30') & (filtered_df['State'] == 'NT')].groupby('Gasdate').sum()

,Demand_x,Supply_x,TransferIn_x,TransferOut_x,Demand_y,Supply_y,TransferIn_y,TransferOut_y,HeldInStorage
Gasdate,,,,,,,,,
2022-11-30,54.012,112.801,44.4,62.31,58.98,105.78,44.044,44.044,0.0


In [52]:
## Formats the demand supply (actuals and forecasts) so that we only have a single row for a single day. Each row has all possible locations demand and supply for that day

print(len(filtered_df.FacilityName.unique()))
print(len(filtered_df.LocationName.unique()))
# print(80 * 21)

looped_fdf = filtered_df[(filtered_df['FacilityName'] == 'APLNG Pipeline') & (df['LocationName'] == 'Curtis Island')].copy()

z = 'APLNG Pipeline'
g = 'Curtis Island'

looped_fdf.rename(columns = {'Demand_x': 'Demand(fc)_' + z + "_" + g , 'Supply_x': 'Supply(fc)_'+ z + "_" + g , 'TransferIn_x' : 'TIn(fc)_' + z + "_" + g,  'TransferOut_x' : 'TOut(fc)_' + z + "_" + g, \
        'Demand_y': 'Demand(a)_' + z + "_" + g , 'Supply_y': 'Supply(a)_'+ z + "_" + g , 'TransferIn_y' : 'TIn(a)_' + z + "_" + g,  'TransferOut_y' : 'TOut(a)_' + z + "_" + g, "HeldInStorage": 'HeldInStorage_' + z + "_" + g },
        inplace = True)

looped_fdf.drop(labels = ["LocationName",'FacilityName', "FacilityType_x", "State_x"], axis = 1, inplace = True )


facility_names = filtered_df.FacilityName.unique()
location_names = filtered_df.LocationName.unique()

i = 0

for z in facility_names: 
    for g in location_names:

        print("Iteration: ", i)
        i += 1

        if ((z == 'APLNG Pipeline') and (g == 'Curtis Island') or (len(filtered_df[(filtered_df['FacilityName'] == z) & (df['LocationName'] == g)]) == 0) ):
            continue

        temp =  filtered_df[(filtered_df['FacilityName'] == z) & (df['LocationName'] == g)].copy()
        temp.rename(columns = {'Demand_x': 'Demand(fc)_' + z + "_" + g , 'Supply_x': 'Supply(fc)_'+ z + "_" + g , 'TransferIn_x' : 'TIn(fc)_' + z + "_" + g,  'TransferOut_x' : 'TOut(fc)_' + z + "_" + g, \
            'Demand_y': 'Demand(a)_' + z + "_" + g , 'Supply_y': 'Supply(a)_'+ z + "_" + g , 'TransferIn_y' : 'TIn(a)_' + z + "_" + g,  'TransferOut_y' : 'TOut(a)_' + z + "_" + g, "HeldInStorage": 'HeldInStorage_' + z + "_" + g },
            inplace = True)
        temp.drop(labels = [ 'LocationName', 'FacilityName', "FacilityType_x", "State_x"], axis = 1, inplace = True )

        # print("lfdf: ", looped_fdf.columns)
        # print("temp: ", temp.columns)


        looped_fdf = looped_fdf.merge(temp, on = 'Gasdate', how = 'outer', suffixes= (None, None))

        
# looped_fdf.fillna(0) ## Uncomment if needed 


80
21
Iteration:  0
Iteration:  1
Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38
Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66

Some data checks
---
- Normalise/Scaling
- Correlation 
- Distribution 
- Outliers


In [ ]:
train_data = 
train_data['Target'] = target

C_mat = train_data.corr()
fig = plt.figure(figsize = (15,15))

sb.heatmap(C_mat, vmax = .8, square = True)
plt.show()

In [ ]:
import pandas as pd;
import matplotlib.pyplot as plt
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RNN, SimpleRNN
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.layers.core import Activation
from keras.callbacks import LambdaCallback
from sklearn.preprocessing import MinMaxScaler